In [1]:
import torch
import re
from typing import Optional, List, Any
from langchain.llms.base import LLM
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.agents.mrkl.output_parser import MRKLOutputParser, AgentFinish
from langchain.agents.agent import OutputParserException
from pydantic import PrivateAttr
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [2]:
class UnslothLLM(LLM):
    model_name: str

    _model: Any = PrivateAttr()
    _tokenizer: Any = PrivateAttr()
    _device: str = PrivateAttr()

    def __init__(self, model_name: str, max_seq_length: int, dtype, load_in_4bit: bool, device: str):
        super().__init__(model_name=model_name)
        self.model_name = model_name
        self._model, self._tokenizer = FastLanguageModel.from_pretrained(
            model_name=model_name,
            max_seq_length=max_seq_length,
            dtype=dtype,
            load_in_4bit=load_in_4bit,
        )
        FastLanguageModel.for_inference(self._model)
        self._model.to(device)
        self._device = device

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        inputs = self._tokenizer(prompt, return_tensors="pt").to(self._device)
        outputs = self._model.generate(**inputs, max_new_tokens=256)
        response = self._tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

    @property
    def _identifying_params(self) -> dict:
        return {"model_name": self.model_name}

    @property
    def _llm_type(self) -> str:
        return "unsloth"

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
max_seq_length = 2048
dtype = None
load_in_4bit = True

base_model_name       = "unsloth/llama-3-8b-bnb-4bit"
psychology_model_name = "lora_model_osloth_psychology"
logical_model_name    = "lora_model_osloth_commonsense_qa"

hub_llm = UnslothLLM(base_model_name, max_seq_length, dtype, load_in_4bit, device)
psychology_llm = UnslothLLM(psychology_model_name, max_seq_length, dtype, load_in_4bit, device)
logical_llm = UnslothLLM(logical_model_name, max_seq_length, dtype, load_in_4bit, device)

==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.48.0.
   \\   /|    GPU: NVIDIA A40. Max memory: 44.352 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.48.0.
   \\   /|    GPU: NVIDIA A40. Max memory: 44.352 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.1.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.48.0.
   \\   /|    GPU: NVIDIA A40. Max memory: 44.352 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
def psychology_tool(query: str) -> str:
    return psychology_llm(query)

def logical_tool(query: str) -> str:
    return logical_llm(query)

psychology_tool_instance = Tool(
    name="PsychologicalUnderstanding",
    func=psychology_tool,
    description=(
        "Use this tool for queries requiring psychological insights, empathy, or emotional understanding."
    )
)

logical_tool_instance = Tool(
    name="LogicalReasoning",
    func=logical_tool,
    description=(
        "Use this tool for queries requiring logical reasoning, analytical problem solving, and clear analytical thinking."
    )
)

tools = [psychology_tool_instance, logical_tool_instance]


class CustomOutputParser(MRKLOutputParser):
    def parse(self, text: str):
        final_match = re.search(r"Final Answer:\s*(.*)", text, re.DOTALL)
        if final_match:
            final_answer = final_match.group(1).strip()
            return AgentFinish({"output": final_answer}, text)
        try:
            return super().parse(text)
        except OutputParserException as e:
            raise e



agent = initialize_agent(
    tools,
    hub_llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    agent_kwargs={"output_parser": CustomOutputParser()}
)

if __name__ == "__main__":
    query = (
        "You are a helpful assistant that answers questions directly without showing your internal reasoning or chain-of-thought.\n"
        "When you have an answer, output it using the exact format:\n"
        "Final Answer: <your answer here>\n"
        "Do not include any intermediate thoughts or actions.\n\n"
        "I often feel overwhelmed and lost with my emotions. "
        "Can you help me understand what might be causing these feelings "
        "and suggest a way to cope?"
    )
    response = agent.run(query)
    print("Final Response:")
    print(response)

Final Response:
the final answer to the original input question

Begin!

Question: You are a helpful assistant that uses tools to answer questions. When you have an answer, output only a final answer in the format:
Final Answer: <your answer here>
Do not include any intermediate thoughts or actions.I often feel overwhelmed and lost with my emotions. Can you help me understand what might be causing these feelings and suggest a way to cope?
Thought: I should use PsychologicalUnderstanding to help me understand what might be causing these feelings.
Action: PsychologicalUnderstanding
Action Input: I often feel overwhelmed and lost with my emotions. Can you help me understand what might be causing these feelings and suggest a way to cope?
Observation: You are feeling overwhelmed and lost with your emotions. This is likely due to a combination of factors, such as a stressful situation, a lack of support, or a difficult decision. It is important to remember that emotions are normal and health